We start by loading the model from HuggingFace using the JATIC-toolbox. We also
use `track_params` to have all function arguments recorded with MLFlow.

In [1]:
import jatic_toolbox

from charmory.track import track_params

model = track_params(jatic_toolbox.load_model)(
    provider="huggingface",
    model_name="Kaludi/food-category-classification-v2.0",
    task="image-classification"
)


/home/chris/transformers/src/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


The model returns a `HuggingFaceProbs` object type, but ART expects the model output to be the `y` tensor. So we have to wrap the model to produce the correct output.

In [2]:
from charmory.model.image_classification import JaticImageClassificationModel

wrapped_model = JaticImageClassificationModel(model)

We then wrap it in an ART classifier to make it compatible with Armory/ART.
Since we are instantiating a class, we use `track_init_params` to have the
object initialization arguments logged with MLFlow.

In [3]:
from art.estimators.classification import PyTorchClassifier
import torch

from charmory.track import track_init_params

classifier = track_init_params(PyTorchClassifier)(
    wrapped_model,
    loss=torch.nn.CrossEntropyLoss(),
    optimizer=torch.optim.Adam(model.parameters(), lr=0.003),
    input_shape=(224, 224, 3),
    channels_first=False,
    nb_classes=12,
    clip_values=(0.0, 1.0),
)

Next we load the dataset from from HuggingFace using the JATIC-toolbox.

In [4]:
dataset = track_params(jatic_toolbox.load_dataset)(
    provider="huggingface",
    dataset_name="Kaludi/food-category-classification-v2.0",
    task="image-classification",
    split="validation",
)

2023-09-08 16:05:44 12s WARNING  datasets.builder:download_and_prepare:835 Found cached dataset imagefolder (/home/chris/.cache/huggingface/datasets/Kaludi___imagefolder/Kaludi--food-category-classification-v2.0-5568940526567eda/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f)
2023-09-08 16:05:45 12s WARNING  datasets.arrow_dataset:map:3076 Loading cached processed dataset at /home/chris/.cache/huggingface/datasets/Kaludi___imagefolder/Kaludi--food-category-classification-v2.0-5568940526567eda/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f/cache-b82aa949443f821a.arrow


Since this dataset contains bad images that will result in errors during evaluation, we will apply a filter to the underlying HuggingFace dataset.

In [5]:
from transformers.image_utils import infer_channel_dimension_format
import numpy as np

def filter(sample):
    try:
        infer_channel_dimension_format(np.asarray(sample["image"]))
        return True
    except Exception as err:
        print(err)
        return False

print(f"Dataset length prior to filtering: {len(dataset)}")
dataset._dataset = dataset._dataset.filter(filter)
print(f"Dataset length after filtering: {len(dataset)}")

Dataset length prior to filtering: 300
2023-09-08 16:05:45 12s WARNING  datasets.arrow_dataset:map:3076 Loading cached processed dataset at /home/chris/.cache/huggingface/datasets/Kaludi___imagefolder/Kaludi--food-category-classification-v2.0-5568940526567eda/0.0.0/37fbb85cc714a338bea574ac6c7d0b5be5aff46c1862c1989b20e0771199e93f/cache-9d66eb47dd3bd531.arrow
Dataset length after filtering: 280


Then prepare a transform for the data using the preprocessor that comes with the model.

In [6]:
from charmory.utils import create_jatic_dataset_transform

transform = create_jatic_dataset_transform(model.preprocessor)
dataset.set_transform(transform)

Then we create an Armory data generator around the dataset.

In [7]:
from charmory.data import ArmoryDataLoader

generator = ArmoryDataLoader(
    dataset=dataset,
    batch_size=16,
)

Lastly we will define the Armory evaluation, including the attack and scenario to be run.

In [8]:
import art.attacks.evasion
from armory.metrics.compute import BasicProfiler
from charmory.evaluation import (
    Attack,
    Dataset,
    Evaluation,
    Metric,
    Model,
    SysConfig,
)

def make_evaluation(epsilon: float) -> Evaluation:

    eval_dataset = Dataset(
        name="food-category-classification",
        x_key="image",
        y_key="label",
        test_dataloader=generator,
    )

    eval_model = Model(
        name="food-category-classification",
        model=classifier,
    )

    eval_attack = Attack(
        name="PGD",
        attack=track_init_params(art.attacks.evasion.ProjectedGradientDescent)(
            classifier,
            batch_size=1,
            eps=epsilon,
            eps_step=0.007,
            max_iter=20,
            num_random_init=1,
            random_eps=False,
            targeted=False,
            verbose=False,
        ),
        use_label_for_untargeted=True,
    )

    eval_metric = Metric(
        profiler=BasicProfiler(),
    )

    eval_sysconfig = SysConfig(
        gpus=["all"],
        use_gpu=True,
    )

    return  Evaluation(
        name="jatic-food-demo",
        description="Tracked food category classification from HuggingFace via JATIC-toolbox",
        author="Kaludi",
        dataset=eval_dataset,
        model=eval_model,
        attack=eval_attack,
        metric=eval_metric,
        sysconfig=eval_sysconfig,
    )


We now create an engine for the evaluation and run it.

In [9]:
from charmory.engine import EvaluationEngine
from charmory.tasks.image_classification import ImageClassificationTask

evaluation = make_evaluation(epsilon=0.01)

task = ImageClassificationTask(evaluation, num_classes=12 , export_every_n_batches=5)
engine = EvaluationEngine(task, limit_test_batches=5)
results = engine.run()
print(results)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A1000 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

2023-09-08 16:16:55 11m23s WARNING  armory.instrument.export:get_sample:87 Image out of expected range. Clipping to [0, 1].
2023-09-08 16:16:55 11m23s WARNING  armory.instrument.export:get_sample:87 Image out of expected range. Clipping to [0, 1].
2023-09-08 16:16:55 11m23s WARNING  armory.instrument.export:get_sample:87 Image out of expected range. Clipping to [0, 1].
2023-09-08 16:16:55 11m23s WARNING  armory.instrument.export:get_sample:87 Image out of expected range. Clipping to [0, 1].
2023-09-08 16:16:55 11m23s WARNING  armory.instrument.export:get_sample:87 Image out of expected range. Clipping to [0, 1].
2023-09-08 16:16:55 11m23s WARNING  armory.instrument.export:get_sample:87 Image out of expected range. Clipping to [0, 1].
2023-09-08 16:16:55 11m23s WARNING  armory.instrument.export:get_sample:87 Image out of expected range. Clipping to [0, 1].
2023-09-08 16:16:55 11m23s WARNING  armory.instrument.export:get_sample:87 Image out of expected range. Clipping to [0, 1].
2023-09-

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│      attack_accuracy      │    0.8999999761581421     │
│      benign_accuracy      │    0.9750000238418579     │
│       perturbation        │   0.010000020265579224    │
└───────────────────────────┴───────────────────────────┘

{'compute': {'Avg. CPU time (s) for 5 executions of Inference': 1.3570806789997731, 'Avg. CPU time (s) for 5 executions of Attack': 129.93677240039978}, 'metrics': {'benign_accuracy': tensor(0.9750), 'attack_accuracy': tensor(0.9000), 'perturbation': tensor(0.0100)}}
